# Lehká analýza

1. Celkový pohled na data 
2. Základní statistiky: min, max, mean, median, standard deviation, korelace atd.
3.  Groupby
4. Statistické testy: test normality, párové testy
5. Lineární regrese

In [1]:
import pandas as pd

AttributeError: module 'numpy' has no attribute 'ndarray'

In [ ]:
pokemon_df = pd.read_csv("../data/Pokemon.csv", index_col="id")

## Základní údaje o datech

Počet řádků a sloupců

In [ ]:
pokemon_df.shape

Přehled o platných a neplatných hodnotách a datových typech.

In [ ]:
pokemon_df.info()

Pomocí metody `count` lze získat stejné údaje jako výše, ale ve strojově přívětivějším formátu.

In [ ]:
pokemon_df.count()

## A teď už nějaká statistika

Metoda `describe` vypíše základní statistické ukazatele: počet, průměr, standardní odchylku, minimum, maximum a kvantily. Výsedek je opět DataFrame, takže na něj můžeme použít vše, co umíme.

In [ ]:
pokemon_df.describe() #.loc["min" : "max"]

Jednotlivé hodnoty můžeme zjistit i přímo, buď pro všechny sloupce najednou.

In [ ]:
pokemon_df.max()

Nebo pro každý zvlášť, pomocí indexování.

In [ ]:
pokemon_df["weight"].max()

In [ ]:
pokemon_df["weight"].mean()

In [ ]:
pokemon_df["weight"].median()
# medián je 50% kvantil

In [ ]:
pokemon_df["weight"].mode()
# mode, česky modus je hodnota s nepočetnějším výskytem 

Pokud chceme vědět, který pokemon je nejtěžší použijeme `idmax`.

In [ ]:
print(pokemon_df["weight"].idxmax())
# pro vypsaní informací použijeme iloc a s indexem z idmax
pokemon_df.iloc[pokemon_df["weight"].idxmax()]

Nebo můžeme dataframe seřadit podle váhy a kouknout se na prvního pokemona.

In [ ]:
pokemon_df.sort_values(by="weight", ascending=False).head()

**Cvičení** Spočítejte průměrnou váhu dospělých Pokémonů lehčích než nějtěžší baby pokémon.

## Groupby
Poslední důležitou operací je groupby ta rozdělí DataFrame podle nějaké kategorické proměnné nebo taky podle funkce do skupin.

In [ ]:
# sloupec 'id' zahodíme aby nepřekážel ve výpisu
pokemon_df.groupby("shape").describe()

In [ ]:
# rozdělení podle funkce:
# funkce se zavolá postupně na každou položku indexu a její výstup se použije k zařazení dané položky do skupiny
def get_weight_category(irow):
    row = pokemon_df.loc[irow]
    if row.weight < 9:
        return "light"
    elif row.weight < 30:
        return "medium"
    else:
        return "heavy"
    
pokemon_df.groupby(by=get_weight_category).describe()

## Statistické testy

Na základní statistiku nám stačí `scipy` a její modul `stats`. Pro pokročilejší analýzu existují například [`statsmodels`](https://www.statsmodels.org/stable/index.html), [`spm`](http://www.spm1d.org/) nebo pro Bayesovskou analýzu a Monte Carlo metody [`pymc`](https://docs.pymc.io/). 

In [ ]:
import scipy.stats as st

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
mpl.rcParams['figure.dpi'] = 100 # bigger figures, yayy!

Test normality

In [ ]:
st.normaltest(pokemon_df["weight"])

In [ ]:
sns.distplot(pokemon_df["weight"], fit=st.norm)

Trochu pokročilejší použití pandas, scipy.stats a matplotlibu: test sloupců na normalitu dat.

In [ ]:
fig, axs = plt.subplots(ncols=len(pokemon_df.loc[:, "hp" : "total"].columns)//2 + 1, nrows=2, figsize=(12, 7), sharey=True)
for col, ax in zip(pokemon_df.loc[:, "hp" : "total"].columns, axs.flatten()):
    
    ntest = st.normaltest(pokemon_df[col])
    
    sns.distplot(pokemon_df[col], fit=st.norm, ax=ax)
    ax.text(ax.get_xlim()[1], ax.get_ylim()[1], "st=%.4f \np=%.4e" % ntest, 
            ha='right', va='top',
            bbox={'facecolor':'red', 'alpha':0.5, 'pad':1} if ntest.pvalue < .005 else None)
axs[-1, -1].axis("off");

Jednoduchý párový test - data nemají normální rozdělení musíme použít nějaký neparametrický třeba Kruskal-Wallis

In [ ]:
st.kruskal(pokemon_df[pokemon_df["shape"] == "quadruped"]["speed"], 
           pokemon_df[pokemon_df["shape"] == "upright"]["speed"])

*Závěr*: Rychlost čtyřnohých a dvounohých Pokémonů se statisticky významně *neliší*.

## Lineární regrese

In [ ]:
sns.regplot(x="speed", y="total", data=pokemon_df)

In [ ]:
st.linregress(pokemon_df["speed"], pokemon_df["total"])